In [1]:
import logging; logging.basicConfig(level=logging.INFO)
import tensorflow as tf
import numpy as np
import logictensornetworks as ltn

In [2]:
a = ltn.proposition(0.2,trainable=True)
b = ltn.proposition(0.5,trainable=True)
c = ltn.proposition(0.5,trainable=True)
w1 = ltn.proposition(0.3)
w2 = ltn.proposition(0.9)

x = ltn.variable("x", np.array([[1,2],[3,4],[5,6]]))
P = ltn.Predicate.MLP(input_shapes=[(2)])

In [3]:
Not = ltn.Wrapper_Connective(ltn.fuzzy_ops.Not_Std())
And = ltn.Wrapper_Connective(ltn.fuzzy_ops.And_Prod())
Or = ltn.Wrapper_Connective(ltn.fuzzy_ops.Or_ProbSum())
Implies = ltn.Wrapper_Connective(ltn.fuzzy_ops.Implies_Reichenbach())
Forall = ltn.Wrapper_Quantifier(ltn.fuzzy_ops.Aggreg_pMeanError(p=5),semantics="forall")
Exists = ltn.Wrapper_Quantifier(ltn.fuzzy_ops.Aggreg_pMean(p=10),semantics="exists")

In [4]:
@tf.function
def axioms():
    axioms = [
        # [ (A and B and (forall x: P(x))) -> Not C ] and C
        And(
            Implies(And(And(a,b),Forall(x,P(x))),
                    Not(c)),
            c
        ),
        # w1 -> (forall x: P(x))
        Implies(w1, Forall(x,P(x))),
        # w2 -> (Exists x: P(x))
        Implies(w2, Exists(x,P(x)))
    ]
    weights = [
        1.,
        1.,
        1.
    ]
    axioms = tf.stack([tf.squeeze(ax) for ax in axioms])
    weights = tf.stack(weights)
    sat_level = tf.reduce_sum(weights*axioms)/tf.reduce_sum(weights)
    return sat_level, axioms

In [5]:
trainable_variables = [a,b,c]
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)

for epoch in range(1000):
    with tf.GradientTape() as tape:
        loss_value = 1. - axioms()[0]
    grads = tape.gradient(loss_value, trainable_variables)
    optimizer.apply_gradients(zip(grads, trainable_variables))
    if epoch%100 == 0:
        print("Epoch %d: Sat Level %.3f"%(epoch, axioms()[0]))
print("Training finished at Epoch %d with Sat Level %.3f"%(epoch, axioms()[0]))

Epoch 0: Sat Level 0.603
Epoch 100: Sat Level 0.704
Epoch 200: Sat Level 0.764
Epoch 300: Sat Level 0.769
Epoch 400: Sat Level 0.772
Epoch 500: Sat Level 0.772
Epoch 600: Sat Level 0.772
Epoch 700: Sat Level 0.772
Epoch 800: Sat Level 0.772
Epoch 900: Sat Level 0.772
Training finished at Epoch 999 with Sat Level 0.772


In [6]:
print([a.numpy(),b.numpy(),c.numpy()])
print(axioms()[1].numpy())

[0.0, 0.45801198, 1.0]
[0.99994075 0.8218677  0.49294233]
